#ICAIL Final Graduation Project_Random Password Maker(G8)

##(1) Importing Libraries

In [ ]:
import random
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support,confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.inspection import permutation_importance
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, HTML

##(2) Data Loading & Preparation :




In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jeffersonvalandro/password-dataset")

print("Path to dataset files:", path)

In [ ]:
data = pd.read_csv('/root/.cache/kagglehub/datasets/jeffersonvalandro/password-dataset/versions/1/passwords_dataset.csv')

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
# Check for missing values
data.isnull().sum()

In [ ]:
data['Strength'].value_counts()

In [ ]:
# We encode to give categories (Weak, Medium, Strong) a numerical value (0, 1, 2) to use for modeling
label_encoder = LabelEncoder()
data['Strength'] = label_encoder.fit_transform(data['Strength'])

##(3) Model Training & Evaluation:

In [ ]:
# I only care about these columns for the independent variables
X = data[['Has Lowercase', 'Has Uppercase', 'Has Special Character', 'Length']]

# Dependent variable
y = data['Strength']

In [ ]:
# Split the data into training and testing sets (80/20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

### Logistic Regression model:

In [ ]:
# Logistic Regression model
log_reg_model = LogisticRegression(max_iter=1000)
log_reg_model.fit(X_train, y_train)

In [ ]:
# Predictions for Logistic Regression
y_pred_log_reg = log_reg_model.predict(X_test)
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)

In [ ]:
# Evaluate additional metrics for Logistic Regression
precision_log_reg, recall_log_reg, f1_log_reg, _ = precision_recall_fscore_support(y_test, y_pred_log_reg, average='weighted')

In [ ]:
# Print the evaluation results
print(f"Logistic Regression Accuracy: {accuracy_log_reg:.4f}")
print(f"Logistic Regression Precision: {precision_log_reg:.4f}")
print(f"Logistic Regression Recall: {recall_log_reg:.4f}")
print(f"Logistic Regression F1-score: {f1_log_reg:.4f}")

In [ ]:
# Cross-validation for Logistic Regression (accuracy)
log_reg_cv_scores = cross_val_score(log_reg_model, X, y, cv=5, scoring='accuracy')  # 5-fold cross-validation

In [ ]:
# Print the cross-validation results
print(f"Logistic Regression Cross-Validation Accuracy: {log_reg_cv_scores.mean():.4f} ± {log_reg_cv_scores.std():.4f}")

### Random Forest model:

In [ ]:
# Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=30)
rf_model.fit(X_train, y_train)

In [ ]:
# Predictions for Random Forest
y_pred_rf = rf_model.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)

In [ ]:
# Evaluate additional metrics for Random Forest
precision_rf, recall_rf, f1_rf, _ = precision_recall_fscore_support(y_test, y_pred_rf, average='weighted')

In [ ]:
# Print the evaluation results
print(f"Random Forest Accuracy: {accuracy_rf:.4f}")
print(f"Random Forest Precision: {precision_rf:.4f}")
print(f"Random Forest Recall: {recall_rf:.4f}")
print(f"Random Forest F1-score: {f1_rf:.4f}")

In [ ]:
# Cross-validation for Random Forest (accuracy)
rf_cv_scores = cross_val_score(rf_model, X, y, cv=5, scoring='accuracy')  # 5-fold cross-validation

In [ ]:
# Print the cross-validation results
print(f"Random Forest Cross-Validation Accuracy: {rf_cv_scores.mean():.4f} ± {rf_cv_scores.std():.4f}")

## (4) Visualizations :

In [ ]:
# Convert 'Strength' back to the original labels for visualization
data['Strength'] = label_encoder.inverse_transform(data['Strength'])

In [ ]:
# Define the order of the categories
strength_order = ['Weak', 'Medium', 'Strong']

In [ ]:
# Create the boxplot with ordered categories
plt.figure(figsize=(8, 6))
sns.boxplot(x='Strength', y='Length', data=data, palette='Set2', order=strength_order)
plt.title("Password Length Distribution by Strength")
plt.xlabel("Password Strength")
plt.ylabel("Password Length")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# A feature graph of what most contributes to a strong password
result = permutation_importance(rf_model, X, y, n_repeats=10, random_state=42)
feature_names = X.columns
feature_importances = result.importances_mean
plt.figure(figsize=(10, 6))
plt.bar(feature_names, feature_importances)
plt.title('What contributes the most to Password Strength?')
plt.xlabel('Features')
plt.ylabel('Importance Score')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## (5) The Interactive Application :

#Random Password Generator

In [ ]:
# 1. Password Strength Checker Class (The class remains almost the same)

class PasswordStrengthChecker:
    def __init__(self, models, label_encoder):
        self.models = models
        self.label_encoder = label_encoder
        self.style_map = {
            'Weak': {'color': 'red', 'emoji': '😞'},
            'Medium': {'color': 'orange', 'emoji': '😐'},
            'Strong': {'color': 'green', 'emoji': '💪'}
        }
        self.create_interface()

    def check_password_strength(self, password):
        features = {
            'Has Lowercase': int(any(c.islower() for c in password)),
            'Has Uppercase': int(any(c.isupper() for c in password)),
            'Has Special Character': int(any(not c.isalnum() for c in password)),
            'Length': len(password)
        }
        X = pd.DataFrame([features])
        predictions = {}
        for name, model in self.models.items():
            pred_num = model.predict(X)[0]
            pred_label = self.label_encoder.inverse_transform([pred_num])[0]
            predictions[name] = pred_label
        return predictions

    def create_interface(self):
        self.password_input = widgets.Password(description='Password:', placeholder='Enter password', layout=widgets.Layout(width='50%'))
        self.output = widgets.Output()
        self.password_input.observe(self.on_password_change, names='value')

    def on_password_change(self, change):
        with self.output:
            self.output.clear_output()
            password = change['new']
            if not password:
                return
            predictions = self.check_password_strength(password)
            display(HTML("<h4>Password Strength Predictions (توقعات قوة كلمة المرور):</h4>"))
            for model, strength in predictions.items():
                style = self.style_map.get(strength, {})
                color = style.get('color', 'black')
                emoji = style.get('emoji', '')
                display(HTML(f"<b>{model.replace('_', ' ').title()}:</b> <span style='color:{color}'>{strength} {emoji}</span>"))

# ---------------------------------------------------------------------------
# 2. Create an instance of the checker first
# ---------------------------------------------------------------------------
checker = PasswordStrengthChecker({'logistic_regression': log_reg_model, 'random_forest': rf_model}, label_encoder)

# ---------------------------------------------------------------------------
# 3. Widget-based Password Generator
# ---------------------------------------------------------------------------

# Input fields for the generator
style = {'description_width': 'initial'}
total_chars = widgets.IntText(value=12, description='Total Characters (العدد الإجمالي):', style=style)
num_letters = widgets.IntText(value=8, description='Number of Letters (عدد الحروف):', style=style)
num_numbers = widgets.IntText(value=2, description='Number of Numbers (عدد الأرقام):', style=style)
num_symbols = widgets.IntText(value=2, description='Number of Symbols (عدد الرموز):', style=style)

# Button to trigger generation
generate_button = widgets.Button(description="Generate & Check Password (أنشئ وافحص كلمة المرور)", button_style='success')
generator_output = widgets.Output() # To show messages and errors

# Function to handle button click
def on_generate_button_clicked(b):
    with generator_output:
        generator_output.clear_output()
        # Validation check
        if total_chars.value != num_letters.value + num_numbers.value + num_symbols.value:
            print("Error: The sum of parts must equal the total number of characters.")
            print("خطأ: مجموع الحروف والأرقام والرموز يجب أن يساوي العدد الإجمالي.")
            return

        # Password generation logic
        letters = random.choices(string.ascii_letters, k=num_letters.value)
        numbers = random.choices(string.digits, k=num_numbers.value)
        symbols = random.choices(string.punctuation, k=num_symbols.value)

        password_list = letters + numbers + symbols
        random.shuffle(password_list)
        generated_password = ''.join(password_list)

        # --- THIS IS THE KEY STEP FOR AUTOMATION ---

        checker.password_input.value = generated_password

        display(HTML(f"<b>New Password Generated (كلمة مرور جديدة):</b> {generated_password}"))


generate_button.on_click(on_generate_button_clicked)

# ---------------------------------------------------------------------------
# 4. Display all widgets together in a structured layout
# ---------------------------------------------------------------------------

# Layout for the generator inputs
generator_inputs = widgets.VBox([
    widgets.HTML("<h3>Password Generator (مولد كلمات المرور)</h3>"),
    total_chars,
    num_letters,
    num_numbers,
    num_symbols,
    generate_button,
    generator_output
])

# Layout for the checker
checker_ui = widgets.VBox([
    widgets.HTML("<hr><h3>Password Strength Checker (فاحص قوة كلمة المرور)</h3>"),
    checker.password_input,
    checker.output
])

# Display everything
display(generator_inputs, checker_ui)